In [29]:
import re
import codecs
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import textstat
import unicodedata
import pandas as pd
import math
import textstat
import numpy as np
import fnmatch
import nlp


from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold

from cluster import cluster_ip, kmeans_cluster, agglomerative_cluster
from lxml import etree
from dataset import generate
from helpers import readFile

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pbonafe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pbonafe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Preparo del espacio de trabajo

In [14]:
actual_directory = os.getcwd()

PATH_CORPUS = os.path.join(actual_directory, "mini_corpus")
PATH_CORPUS

'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus'

In [15]:
# Produce una lista con los nombres de los archivos txt del corpus
textos_originales = glob.glob(os.path.join(PATH_CORPUS, "*.txt"))
textos_originales

['c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00001.txt',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00002.txt',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00003.txt',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00004.txt',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00005.txt',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00006.txt',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00007.txt',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00008.txt',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00009.txt',
 

In [16]:
# Creo la carpeta donde voy a guardar los archivos limpios
cleaned_corpus = os.path.join(actual_directory, "cleaned_corpus")

if not os.path.exists( cleaned_corpus):
    os.mkdir(cleaned_corpus)

In [17]:
# De esta manera le voy robo el nombre a los xml
path_xml = os.path.join(actual_directory, "mini_corpus")
path_xml = os.path.join( path_xml, "*.xml")
xml_docs = glob.glob(path_xml)
xml_docs

['c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00001.xml',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00002.xml',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00003.xml',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00004.xml',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00005.xml',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00006.xml',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00007.xml',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00008.xml',
 'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB\\mini_corpus\\suspicious-document00009.xml',
 

In [18]:
# Creo la carpeta donde voy a guardar los archivos sospechosos
sospechosos_path = os.path.join(actual_directory, "sospechoso")

if not os.path.exists(sospechosos_path):
    os.mkdir(sospechosos_path)
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos")
    patron_inicio = 'sosp*'
    patron_final = '*.txt'
    archivos = os.listdir(sospechosos_path)
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_inicio) and fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(sospechosos_path, archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")

Se elimino carpeta vieja de fragmentos sospechosos


In [19]:
output_xml = os.path.join(actual_directory, "xml")
    
if not os.path.exists(output_xml):
    os.mkdir(output_xml)
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos en formato xml")
    patron_final = '*.xml'
    archivos = os.listdir(output_xml)
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(output_xml, archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")

Se elimino carpeta vieja de fragmentos sospechosos en formato xml


## Probamos con un solo archivo

In [8]:
# Nombre de Archivo
filename = os.path.basename(textos_originales[0])

# Separo nombre de archivo y extension
filename2, extension = os.path.splitext(filename)

filename, filename2, extension

('suspicious-document00001.txt', 'suspicious-document00001', '.txt')

In [9]:
# Leemos el archivo
text = readFile( os.path.join( PATH_CORPUS, filename))
text

'The convening of this body gave rise to much dissatisfaction among the people of Texas.  They\nhad assumed that affairs were to go on as of old, and that the reintegration of the State was\nto take place under the administration of Governor Murray, who, meanwhile, had taken it upon\nhimself, together with the Legislature, to authorize the election of delegates to a State Convention,\nwithout restriction as to who should be entitled to vote.  Thus encouraged, the element but\nlately in armed rebellion was now fully bent on restoring the State to the Union without any\nintervention whatever of the Federal Government; but the advent of Hamilton put an end to such\nillusions, since his proclamation promptly disfranchised the element in question, whose consequent\ndisappointment and chagrin were so great as to render this factor of the community almost uncontrollable.\nThe provisional Governor at once rescinded the edict of Governor Murray, prohibited the assembling\nof his convention, and

# Segmentación

In [10]:
from segmentador import Segmentador

data_list = []
length = [] 

segmenter  = Segmentador(text)

### Segmentamos por oraciones

In [11]:
# La variable data_text posee el texto completo, separado en párrafos
data_text = segmenter.sent_segmentation()

data_text

['The convening of this body gave rise to much dissatisfaction among the people of Texas.',
 'They\nhad assumed that affairs were to go on as of old, and that the reintegration of the State was\nto take place under the administration of Governor Murray, who, meanwhile, had taken it upon\nhimself, together with the Legislature, to authorize the election of delegates to a State Convention,\nwithout restriction as to who should be entitled to vote.',
 'Thus encouraged, the element but\nlately in armed rebellion was now fully bent on restoring the State to the Union without any\nintervention whatever of the Federal Government; but the advent of Hamilton put an end to such\nillusions, since his proclamation promptly disfranchised the element in question, whose consequent\ndisappointment and chagrin were so great as to render this factor of the community almost uncontrollable.',
 'The provisional Governor at once rescinded the edict of Governor Murray, prohibited the assembling\nof his conve

### Segmentamos por parrafos

In [12]:
data_text = segmenter.para_segmentation()
data_text

["The convening of this body gave rise to much dissatisfaction among the people of Texas.  They\nhad assumed that affairs were to go on as of old, and that the reintegration of the State was\nto take place under the administration of Governor Murray, who, meanwhile, had taken it upon\nhimself, together with the Legislature, to authorize the election of delegates to a State Convention,\nwithout restriction as to who should be entitled to vote.  Thus encouraged, the element but\nlately in armed rebellion was now fully bent on restoring the State to the Union without any\nintervention whatever of the Federal Government; but the advent of Hamilton put an end to such\nillusions, since his proclamation promptly disfranchised the element in question, whose consequent\ndisappointment and chagrin were so great as to render this factor of the community almost uncontrollable.\nThe provisional Governor at once rescinded the edict of Governor Murray, prohibited the assembling\nof his convention, an

### Segmentamos por palabras

Poco últi la verdad a primera vista...

In [13]:
data_text = segmenter.word_segmentation()
data_text

['The',
 'convening',
 'of',
 'this',
 'body',
 'gave',
 'rise',
 'to',
 'much',
 'dissatisfaction',
 'among',
 'the',
 'people',
 'of',
 'Texas',
 '.',
 'They',
 'had',
 'assumed',
 'that',
 'affairs',
 'were',
 'to',
 'go',
 'on',
 'as',
 'of',
 'old',
 ',',
 'and',
 'that',
 'the',
 'reintegration',
 'of',
 'the',
 'State',
 'was',
 'to',
 'take',
 'place',
 'under',
 'the',
 'administration',
 'of',
 'Governor',
 'Murray',
 ',',
 'who',
 ',',
 'meanwhile',
 ',',
 'had',
 'taken',
 'it',
 'upon',
 'himself',
 ',',
 'together',
 'with',
 'the',
 'Legislature',
 ',',
 'to',
 'authorize',
 'the',
 'election',
 'of',
 'delegates',
 'to',
 'a',
 'State',
 'Convention',
 ',',
 'without',
 'restriction',
 'as',
 'to',
 'who',
 'should',
 'be',
 'entitled',
 'to',
 'vote',
 '.',
 'Thus',
 'encouraged',
 ',',
 'the',
 'element',
 'but',
 'lately',
 'in',
 'armed',
 'rebellion',
 'was',
 'now',
 'fully',
 'bent',
 'on',
 'restoring',
 'the',
 'State',
 'to',
 'the',
 'Union',
 'without',
 'an

### Segmentamos por tokens

In [16]:
data_text = segmenter.content_segmentation()
data_text

["The convening of this body gave rise to much dissatisfaction among the people of Texas.  They\nhad assumed that affairs were to go on as of old, and that the reintegration of the State was\nto take place under the administration of Governor Murray, who, meanwhile, had taken it upon\nhimself, together with the Legislature, to authorize the election of delegates to a State Convention,\nwithout restriction as to who should be entitled to vote.  Thus encouraged, the element but\nlately in armed rebellion was now fully bent on restoring the State to the Union without any\nintervention whatever of the Federal Government; but the advent of Hamilton put an end to such\nillusions, since his proclamation promptly disfranchised the element in question, whose consequent\ndisappointment and chagrin were so great as to render this factor of the community almost uncontrollable.\nThe provisional Governor at once rescinded the edict of Governor Murray, prohibited the assembling\nof his convention, an

### Segmentamos por grupos de oraciones

In [24]:
# data_text = segmenter.custom_segmentation()
data_text = segmenter.sentence_segmentation()
data_text

['The convening of this body gave rise to much dissatisfaction among the people of Texas.They\nhad assumed that affairs were to go on as of old, and that the reintegration of the State was\nto take place under the administration of Governor Murray, who, meanwhile, had taken it upon\nhimself, together with the Legislature, to authorize the election of delegates to a State Convention,\nwithout restriction as to who should be entitled to vote.Thus encouraged, the element but\nlately in armed rebellion was now fully bent on restoring the State to the Union without any\nintervention whatever of the Federal Government; but the advent of Hamilton put an end to such\nillusions, since his proclamation promptly disfranchised the element in question, whose consequent\ndisappointment and chagrin were so great as to render this factor of the community almost uncontrollable.The provisional Governor at once rescinded the edict of Governor Murray, prohibited the assembling\nof his convention, and shor

In [11]:
  
f = {}
# Comienzo a leer el conjunto de textos
for i in range(len(textos_originales)):
    print("File: " + str(i+1))

    # Leemos el archivo
    text = readFile( os.path.join( PATH_CORPUS, textos_originales[i]))

    
    # La variable data_text posee el texto completo
    data_text = segmenter.sentence_segmentation()

    data_list = []
    length = []
    
    # Se tiene guardar el texto sin limpiar
    for segmento in data_text:
        # A cada segmento se le borra el salto de línea
        segmento = re.sub("\n", " ", segmento)
        # Se guarda la longitud del segmento
        longitud = len(segmento)
        length.append(longitud)
        # Se guarda el segmento
        data_list.append(segmento)
    
    print(data_list)
    # De aqui empezar a procesar la limpieza de texto 
    # Some code...
    # .....
    # .....
    
    

File: 1
['The convening of this body gave rise to much dissatisfaction among the people of Texas.They had assumed that affairs were to go on as of old, and that the reintegration of the State was to take place under the administration of Governor Murray, who, meanwhile, had taken it upon himself, together with the Legislature, to authorize the election of delegates to a State Convention, without restriction as to who should be entitled to vote.Thus encouraged, the element but lately in armed rebellion was now fully bent on restoring the State to the Union without any intervention whatever of the Federal Government; but the advent of Hamilton put an end to such illusions, since his proclamation promptly disfranchised the element in question, whose consequent disappointment and chagrin were so great as to render this factor of the community almost uncontrollable.The provisional Governor at once rescinded the edict of Governor Murray, prohibited the assembling of his convention, and short

## Procesamos los textos originales

In [22]:
actual_directory

'c:\\Users\\pbonafe\\Diplo\\mentoria\\M11-DeteccionPlagio-GrupoB'

In [33]:
# Comienzo a leer el conjunto de textos
for i, archive in enumerate(textos_originales):
    
    print("File: " + str(i+1))
    
    f = open(archive, encoding="utf-8-sig")
    # Leo contenido del archivo
    text = f.read()

    # Cierro el txt
    f.close()           
    
    # Nombre de Archivo
    filename = os.path.basename(archive)
    # Separo nombre de archivo y extension
    filename2, extension = os.path.splitext(filename)
    
    #Seg = Segmentation(text)
    # La variable data_text posee el texto completo, separado en párrafos
    #data_text = Seg.paraSegmentation()

    data_text_parraf = segmenter.para_segmentation()
    data_list = []

    lista_to_df = []
    
    for j, segmento in enumerate(data_text_parraf):

        data_list.append([j, re.sub("\n", " ", segmento), len(re.sub("\n", " ", segmento))])

        lista_to_df.append(
            [j, filename, nlp.getnumOfPunctN(segmento), nlp.gettypeToken(segmento, language="English")]
        )                        
    
    df = pd.DataFrame(lista_to_df, columns=["index", "filename", "getnumOfPunctN", "gettypeToken"])
    
    
    #df["getfleshReadingEase"] = df["getfleshReadingEase"] / df["getfleshReadingEase"].max()
    
    # Seleccionar los valores numéricos del df
    # Este paso se realiza para ir viendo que genera cada párrafo del txt y como fue convertido a un vector
    dataframe = df.iloc[:,[2,3]]
    
    # Aprendizaje No Supervisado
    cluster = cluster_ip(dataframe, 4, "cosine")
    labels = cluster.labels_    
    df["labels"] = labels
    
    realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
    clusterNum=len(set(labels))
    n_noise = list(labels).count(-1) 
    
    print("Estimated number of clusters: %d" % realClusterNum)
    print("Estimated number of noise points: %d" % n_noise)
    counts = df["labels"].value_counts().to_dict()
    print(counts)
    
    # Me sirve para guardar los casos de plagio y compararlos
    xml_name = os.path.basename(xml_docs[i])
    xml_name2, extension = os.path.splitext(xml_name)
    
    # Se crea el archivo txt donde se guardaran los fragmentos sospechosos
    sherlock = codecs.open(sospechosos_path + "sosp" + filename,"w","utf-8")
    
    # Los fragmentos que fueron clasificados como plagio
    # Se guardan en el archivo .txt
    # Tener en cuenta que el dataframe y la lista data_list, manejan los mismos índices
    # Por eso, si en el dataframe se encuentra un "-1" es porque este es un outlier
    for x in range(len(df["labels"])):
        l = df["labels"][x]
        if l == -1:
            print(data_list[x][1])
            print(x)
            sherlock.write(data_list[x][1]+os.linesep)       
               
    sherlock.close()    
    
    document_el = etree.Element("document", reference = xml_name2 + ".txt")

    offset = []

    init = 0

    sum_length = []

    last = -1

    for y in range(len(df["labels"])):       

        l = df["labels"][y]

        if l == -1:

            print(y)

            if y == 0:

                sum_length.append([init, data_list[y][2]])

                print(f"Case 0, y = 0")

            elif y != 0 and (y == (last + 1) or len(sum_length) == 0):

                sum_length.append([init, data_list[y][2]])

                print(f"Case 1, y == (last + 1)")

            elif y != 0 and y != (last + 1):

                this_length = sum([l[1] for l in sum_length])

                feature_el = etree.SubElement(document_el, "feature", name="detected-plagiarism", this_offset=str(sum_length[0][0]), this_length=str(this_length))

                sum_length.clear()

                print(sum_length)

                sum_length.append([init, data_list[y][2]])

                print(f"Case 2, y != (last + 1)")   

            last = y

        if y == (len(df["labels"]) - 1) and len(sum_length) > 0:

            print("Entro2")

            this_length = sum([l[1] for l in sum_length])

            feature_el = etree.SubElement(document_el, "feature", name="detected-plagiarism", this_offset=str(sum_length[0][0]), this_length=str(this_length))


        init += data_list[y][2] + 1





    et = etree.ElementTree(document_el)
    with open(output_xml+xml_name, "wb") as fe:
        et.write(fe, encoding="utf-8", xml_declaration=None, pretty_print=True)

File: 1
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
File: 2
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
File: 3
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
File: 4
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
File: 5
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
File: 6
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
File: 7
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
File: 8
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
File: 9
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
File: 10
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 62}
